<a href="https://colab.research.google.com/github/viveksahukar/sih/blob/main/model_build_train_results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This notebook details how the 3 CNN are trained on 1 Axial and 2 coronal slices, Dicom files are directly fed to CNN.

In [ ]:
random_state = 3 # Set random state
from vs165_modules import * # Load required libraries

# Running model on axial slices

In [ ]:
# Set device
train_on_gpu = cuda.is_available()
device = torch.device("cuda" if train_on_gpu else "cpu")

In [ ]:
# Set hyperparameters for axial model
n_classes = 2 # 2 classes positive and negative
learning_rate_ax = 1e-5 
batch_size_ax = 10 

In [ ]:
ax_train_dataset = DicomDataset('ax_train_new.csv')
ax_valid_dataset = DicomDataset('ax_val_new.csv')
ax_test_dataset = DicomDataset('ax_test_new.csv')

In [ ]:
# train_set, valid_set = torch.utils.data.random_split(dataset , [149, 64])
ax_train_loader = DataLoader(dataset=ax_train_dataset, batch_size=batch_size_ax, shuffle=True)
ax_valid_loader = DataLoader(dataset=ax_valid_dataset, batch_size=batch_size_ax, shuffle=True)

In [ ]:
model_ax = torchvision.models.resnet50(pretrained=True)

n_inputs_ax = model_ax.fc.in_features


model_ax.fc = nn.Sequential(
                      nn.Linear(n_inputs_ax, 256), 
                      nn.ReLU(), 
                      nn.Dropout(0.4),
                      nn.Linear(256, n_classes),                   
                      nn.LogSoftmax(dim=1))

criterion = nn.NLLLoss()
optimizer = optim.Adam(model_ax.parameters(), lr=learning_rate_ax)

save_file_name = f'resnet50-transfer-ax.pt'
checkpoint_path = f'resnet50-transfer-ax.pth'

model_ax.to(device);

In [ ]:
model_ax, history_ax = model_train(
    model_ax,
    criterion,
    optimizer,
    ax_train_loader,
    ax_valid_loader,
    save_file_name=save_file_name,
    max_epochs_stop=50,
    n_epochs=50,
    print_every=10)

In [ ]:
train_loader_ax = DataLoader(dataset=ax_train_dataset, batch_size=1, shuffle=True)
valid_loader_ax = DataLoader(dataset=ax_valid_dataset, batch_size=1, shuffle=True)
test_loader_ax = DataLoader(dataset=ax_test_dataset, batch_size=1, shuffle=True)

df_pred_ax_train = model_eval(model_ax, train_loader_ax)
df_pred_ax_valid = model_eval(model_ax, valid_loader_ax)
df_pred_ax_test = model_eval(model_ax, test_loader_ax)

In [ ]:
loss_accuracy(history_ax, 'Axial')
roc_pr(df_pred_ax_valid, 'Axial', 'Valid Set')
roc_pr(df_pred_ax_test, 'Axial', 'Test Set')

# Now training model on coronal slices 

In [ ]:
cor_1_train_dataset = DicomDataset('cor_val_1.csv')
cor_1_test_dataset = DicomDataset('cor_test_1.csv')
cor_2_train_dataset = DicomDataset('cor_train_2.csv')
cor_2_valid_dataset = DicomDataset('cor_val_2.csv')
cor_2_test_dataset = DicomDataset('cor_test_2.csv')

In [ ]:
n_classes = 2
learning_rate_cor = 1e-5
batch_size_cor = 10

In [ ]:
# train_set, valid_set = torch.utils.data.random_split(dataset , [149, 64])
cor_1_train_loader = DataLoader(dataset=cor_1_train_dataset, batch_size=batch_size_cor, shuffle=True)
cor_1_valid_loader = DataLoader(dataset=cor_1_valid_dataset, batch_size=batch_size_cor, shuffle=True)
cor_1_test_loader = DataLoader(dataset=cor_1_test_dataset, batch_size=batch_size_cor, shuffle=True)
cor_2_train_loader = DataLoader(dataset=cor_2_train_dataset, batch_size=batch_size_cor, shuffle=True)
cor_2_valid_loader = DataLoader(dataset=cor_2_valid_dataset, batch_size=batch_size_cor, shuffle=True)
cor_2_test_loader = DataLoader(dataset=cor_2_test_dataset, batch_size=batch_size_cor, shuffle=True)

In [ ]:
model_cor_1 = torchvision.models.resnet50(pretrained=True)
n_inputs_cor_1 = model_cor_1.fc.in_features
model_cor_1.fc = nn.Sequential(
                      nn.Linear(n_inputs_cor_1, 256), 
                      nn.ReLU(), 
                      nn.Dropout(0.4),
                      nn.Linear(256, n_classes),                   
                      nn.LogSoftmax(dim=1))

model_cor_2 = torchvision.models.resnet50(pretrained=True)
n_inputs_cor_2 = model_cor_2.fc.in_features
model_cor_2.fc = nn.Sequential(
                      nn.Linear(n_inputs_cor_2, 256), 
                      nn.ReLU(), 
                      nn.Dropout(0.4),
                      nn.Linear(256, n_classes),                   
                      nn.LogSoftmax(dim=1))

model_cor_1.to(device);
model_cor_2.to(device);

## Training model on coronal slices 1

In [ ]:
train_on_gpu = cuda.is_available()
criterion = nn.NLLLoss()
optimizer = optim.Adam(model_cor_1.parameters(), lr=learning_rate_cor)

save_file_name = f'resnet50-transfer-cor-1.pt'
checkpoint_path = f'resnet50-transfer-cor-1.pth'

In [ ]:
model_cor_1, history_cor_1 = model_train(
    model_cor_1,
    criterion,
    optimizer,
    cor_1_train_loader,
    cor_1_valid_loader,
    save_file_name=save_file_name,
    max_epochs_stop=50,
    n_epochs=50,
    print_every=10)

In [ ]:
train_loader_cor_1 = DataLoader(dataset=cor_1_train_dataset, batch_size=1, shuffle=True)
valid_loader_cor_1 = DataLoader(dataset=cor_1_valid_dataset, batch_size=1, shuffle=True)
test_loader_cor_1 = DataLoader(dataset=cor_1_test_dataset, batch_size=1, shuffle=True)

df_pred_cor_1_train = model_eval(model_cor_1, train_loader_cor_1)
df_pred_cor_1_valid = model_eval(model_cor_1, valid_loader_cor_1)
df_pred_cor_1_test = model_eval(model_cor_1, test_loader_cor_1)

In [ ]:
loss_accuracy(history_cor_1, 'Coronal 1')
roc_pr(df_pred_cor_1_valid, 'Coronal 1', 'Valid Set')
roc_pr(df_pred_cor_1_test, 'Coronal 1', 'Test Set')

## Training model on coronal slices 2

In [ ]:
train_on_gpu = cuda.is_available()

criterion = nn.NLLLoss()
optimizer = optim.Adam(model_cor_2.parameters(), lr=learning_rate_cor)

save_file_name = f'resnet50-transfer-cor-2.pt'
checkpoint_path = f'resnet50-transfer-cor-2.pth'

In [ ]:
model_cor_2, history_cor_2 = model_train(
    model_cor_2,
    criterion,
    optimizer,
    cor_2_train_loader,
    cor_2_valid_loader,
    save_file_name=save_file_name,
    max_epochs_stop=50,
    n_epochs=50,
    print_every=10)

In [ ]:
train_loader_cor_2 = DataLoader(dataset=cor_2_train_dataset, batch_size=1, shuffle=True)
valid_loader_cor_2 = DataLoader(dataset=cor_2_valid_dataset, batch_size=1, shuffle=True)
test_loader_cor_2 = DataLoader(dataset=cor_2_test_dataset, batch_size=1, shuffle=True)

df_pred_cor_2_train = model_eval(model_cor_2, train_loader_cor_2)
df_pred_cor_2_valid = model_eval(model_cor_2, valid_loader_cor_2)
df_pred_cor_2_test = model_eval(model_cor_2, test_loader_cor_2)

In [ ]:
loss_accuracy(history_cor_2, 'Coronal 2')
roc_pr(df_pred_cor_2_valid, 'Coronal 2', 'Valid Set')
roc_pr(df_pred_cor_2_test, 'Coronal 2', 'Test Set')

## Ensembling the above 3 models using logistic regression model

In [ ]:
df_pred_train = df_pred(df_pred_ax_train, df_pred_cor_1_train, df_pred_cor_2_train)
df_pred_valid = df_pred(df_pred_ax_valid, df_pred_cor_1_valid, df_pred_cor_2_valid)
df_pred_test = df_pred(df_pred_ax_test, df_pred_cor_1_test, df_pred_cor_2_test)

In [ ]:
X_train = df_pred_train[['p_ax', 'p_cor_1', 'p_cor_2']]
y_train = df_pred_train['y_true']

X_valid = df_pred_valid[['p_ax', 'p_cor_1', 'p_cor_2']]
y_valid = df_pred_valid['y_true']

X_test = df_pred_test[['p_ax', 'p_cor_1', 'p_cor_2']]
y_test = df_pred_test['y_true']

In [ ]:
clf = LogisticRegression(random_state=random_state).fit(X_train, y_train)

In [ ]:
# prob score from logistic regression model
p_lgr_valid = clf.predict_proba(X_valid)[:, 1]
p_lgr_test = clf.predict_proba(X_test)[:, 1]

In [ ]:
df_pred_valid['prob'] = p_lgr_valid
df_pred_test['prob'] = p_lgr_test

In [ ]:
roc_pr(df_pred_valid, 'Ensemble', 'Valid Set')
roc_pr(df_pred_test, 'Ensemble', 'Test Set')